In [3]:
import os
import sys
import torch
import numpy as np
import torch.nn as nn
from tqdm import tqdm
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# Get current working directory instead of __file__
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
from RepVGGModel import RepVGGA1WheatModel
from dataLoaderFunc import loadSplitData, createLoader

c:\Users\pacha\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=10, device='cuda'):
    best_val_loss = float('inf')

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0

        for batch_idx, (rgb_batch, dsm_batch, label_batch) in enumerate(train_loader):
            rgb_batch = rgb_batch.to(device)
            dsm_batch = dsm_batch.to(device)
            label_batch = label_batch.to(device)

            optimizer.zero_grad()
            outputs = model(rgb_batch, dsm_batch)
            loss = criterion(outputs, label_batch)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

            if batch_idx % 20 == 0:
                print(f"Epoch {epoch+1}/{num_epochs} | Batch {batch_idx}/{len(train_loader)} | Loss: {loss.item():.4f}")

        train_loss /= len(train_loader)

        # ✅ Validation phase
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for rgb_batch, dsm_batch, label_batch in val_loader:
                rgb_batch = rgb_batch.to(device)
                dsm_batch = dsm_batch.to(device)
                label_batch = label_batch.to(device)

                outputs = model(rgb_batch, dsm_batch)
                loss = criterion(outputs, label_batch)
                val_loss += loss.item()

        val_loss /= len(val_loader)
        scheduler.step(val_loss)

        print(f"✅ Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

        # ✅ Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), "RepVGG_wheat_model.pth")
            print("💾 Saved best model!")

In [5]:
train_df, val_df, test_df = loadSplitData("RGB_DSM_totEarNum.csv")
train_loader, val_loader, test_loader = createLoader(train_df, val_df, test_df)

Train Size: 47840, Validation Size: 5980, Test Size: 5980
Train Batches: 2990, Validation Batches: 374, Test Batches: 374


In [6]:

# ✅ Initialize Model, Loss Function, and Optimizer

# ✅ Universal device selection (works on both Mac M2 & Windows with RTX 4060)
if torch.backends.mps.is_available():
    device = "mps"  # ✅ Use Apple Metal (Mac M1/M2)
    torch.set_default_tensor_type(torch.FloatTensor)
elif torch.cuda.is_available():
    device = "cuda"  # ✅ Use NVIDIA CUDA (Windows RTX 4060)
else:
    device = "cpu"  # ✅ Default to CPU if no GPU is available
print(f"✅ Using device: {device}")

model = RepVGGA1WheatModel().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=2, factor=0.5, verbose=True)

✅ Using device: cuda


c:\Users\pacha\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [7]:
train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=10, device=device)


Epoch 1/10 | Batch 0/2990 | Loss: 111679.5156
Epoch 1/10 | Batch 20/2990 | Loss: 27928.7891
Epoch 1/10 | Batch 40/2990 | Loss: 16225.0430
Epoch 1/10 | Batch 60/2990 | Loss: 10313.7051
Epoch 1/10 | Batch 80/2990 | Loss: 6976.4771
Epoch 1/10 | Batch 100/2990 | Loss: 6770.9893
Epoch 1/10 | Batch 120/2990 | Loss: 7919.0728
Epoch 1/10 | Batch 140/2990 | Loss: 5556.3730
Epoch 1/10 | Batch 160/2990 | Loss: 6362.4482
Epoch 1/10 | Batch 180/2990 | Loss: 3531.2410
Epoch 1/10 | Batch 200/2990 | Loss: 5182.3472
Epoch 1/10 | Batch 220/2990 | Loss: 4948.5356
Epoch 1/10 | Batch 240/2990 | Loss: 3508.9226
Epoch 1/10 | Batch 260/2990 | Loss: 4746.1875
Epoch 1/10 | Batch 280/2990 | Loss: 3269.0288
Epoch 1/10 | Batch 300/2990 | Loss: 5247.0156
Epoch 1/10 | Batch 320/2990 | Loss: 3750.0146
Epoch 1/10 | Batch 340/2990 | Loss: 9968.5664
Epoch 1/10 | Batch 360/2990 | Loss: 2825.5229
Epoch 1/10 | Batch 380/2990 | Loss: 2599.9395
Epoch 1/10 | Batch 400/2990 | Loss: 3431.1045
Epoch 1/10 | Batch 420/2990 | Loss: